In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')

%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()


'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


##### 변수명
* REG_YYMM 년월

* CARD_SIDO_NM 카드이용지역_시도 (가맹점 주소 기준)

* CARD_CCG_NM 카드이용지역_시군구 (가맹점 주소 기준)

* STD_CLSS_NM 업종명

* HOM_SIDO_NM 거주지역_시도 (고객 집주소 기준)

* HOM_CCG_NM 거주지역_시군구 (고객 집주소 기준)

* AGE 연령대

* SEX_CTGO_CD 성별 (1: 남성, 2: 여성)

* FLC 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)

* CSTMR_CNT 이용고객수 (명)

* AMT 이용금액 (원)

* CNT 이용건수 (건)


In [2]:
data = pd.read_csv('train.csv')
sub = pd.read_csv('submission.csv')

In [3]:
data.loc[data.CARD_CCG_NM.isna(),'CARD_CCG_NM'] = '세종'
data.loc[data.HOM_CCG_NM.isna(),'HOM_CCG_NM'] = '세종'

In [4]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [5]:
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,0
1,1,202004,강원,골프장 운영업,0
2,2,202004,강원,과실 및 채소 소매업,0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0


##### sub에는 train에 있는 시군구, 소비자 변수가 없고 직종만 있다 > 지역, 직종별 특성을 찾아야 한다.

In [6]:
city_sum= data.groupby(['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['AMT','CNT','CSTMR_CNT'].sum().reset_index()
city_sum['DANGOL'] = city_sum['CNT']/city_sum['CSTMR_CNT']
city_sum.drop(['CNT','CSTMR_CNT'],axis= 1, inplace = True)

In [7]:
a = sub[['CARD_SIDO_NM','STD_CLSS_NM']].copy()
a.set_index(['CARD_SIDO_NM','STD_CLSS_NM'],inplace = True)
city_sum.set_index(['CARD_SIDO_NM','STD_CLSS_NM'],inplace = True)
city_sum = city_sum.merge(a,left_index=True, right_index=True, how = 'right').reset_index()
city_sum = city_sum.drop_duplicates().sort_values(by = 'REG_YYMM').dropna()

In [8]:
city_sum.head()

,CARD_SIDO_NM,STD_CLSS_NM,REG_YYMM,CARD_CCG_NM,AMT,DANGOL
0,강원,건강보조식품 소매업,201901.0,강릉시,24027180.0,1.056338
82418,경북,여관업,201901.0,성주군,1273000.0,1.434783
82419,경북,여관업,201901.0,안동시,36386640.0,1.421203
82420,경북,여관업,201901.0,영덕군,10176509.0,1.120000
82421,경북,여관업,201901.0,영양군,780000.0,1.300000


In [9]:
new_data = data.groupby(['CARD_SIDO_NM','CARD_CCG_NM','HOM_SIDO_NM','HOM_CCG_NM','REG_YYMM','STD_CLSS_NM'])['AMT','CNT'].sum().reset_index()

In [10]:
lst = [0] * len(new_data)
a = list(new_data['CARD_SIDO_NM'])
b = list(new_data['HOM_SIDO_NM'])
for i in range(len(a)):
    if a[i] != b[i]:
        lst[i] = 1
new_data['LOCAL_CHARGE'] = lst
new_data.drop(['HOM_SIDO_NM','HOM_CCG_NM'],axis =1,inplace = True)

In [11]:
local_sobi = new_data.groupby(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','REG_YYMM','LOCAL_CHARGE'])['AMT','CNT'].sum()
local_sobi2 = local_sobi.unstack().reset_index()
local_sobi2 = local_sobi2.fillna(0)
local_sobi2['stranger_AMT_Percent'] = local_sobi2['AMT'][1]/(local_sobi2['AMT'][0] + local_sobi2['AMT'][1]) * 100
local_sobi2['local_AMT_Percent'] = local_sobi2['AMT'][0]/(local_sobi2['AMT'][0] + local_sobi2['AMT'][1]) * 100
local_sobi2['total_AMT'] = local_sobi2['AMT'][0]+local_sobi2['AMT'][1]
local_sobi2['stranger_CNT'] = local_sobi2['CNT'][1]
local_sobi2['local_CNT'] = local_sobi2['CNT'][0]
local_sobi2['total_CNT'] = local_sobi2['CNT'][0]+local_sobi2['CNT'][1]

In [12]:
local_sobi2.head()

CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM REG_YYMM         AMT       \
LOCAL_CHARGE                                                         0    1   
0                      강원         강릉시  건강보조식품 소매업   201901  24027180.0  0.0   
1                      강원         강릉시  건강보조식품 소매업   201902  26313410.0  0.0   
2                      강원         강릉시  건강보조식품 소매업   201903  21539495.0  0.0   
3                      강원         강릉시  건강보조식품 소매업   201904  16889940.0  0.0   
4                      강원         강릉시  건강보조식품 소매업   201905  20443543.0  0.0   

                CNT      stranger_AMT_Percent local_AMT_Percent   total_AMT  \
LOCAL_CHARGE      0    1                                                      
0             150.0  0.0                  0.0             100.0  24027180.0   
1             174.0  0.0                  0.0             100.0  26313410.0   
2             126.0  0.0                  0.0             100.0  21539495.0   
3             112.0  0.0                  0.0             100.0  16889940.0   
4             141.0  0.0                  0.0             100.0  20443543.0   

             stranger_CNT local_CNT total_CNT  
LOCAL_CHARGE                                   
0                     0.0     150.0     150.0  
1                     0.0     174.0     174.0  
2                     0.0     126.0     126.0  
3                     0.0     112.0     112.0  
4                     0.0     141.0     141.0

In [13]:
new_local_sobi = local_sobi2.set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','REG_YYMM']).iloc[:,-6:]

In [14]:
new_local_sobi

stranger_AMT_Percent  \
LOCAL_CHARGE                                                           
CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM   REG_YYMM                        
강원           강릉시         건강보조식품 소매업    201901               0.000000   
                                       201902               0.000000   
                                       201903               0.000000   
                                       201904               0.000000   
                                       201905               0.000000   
                                       201906               0.000000   
                                       201907               0.000000   
                                       201908               0.000000   
                                       201909               0.000000   
                                       201910               0.000000   
                                       201911               0.000000   
                                       201912               0.000000   
                                       202001               0.000000   
                                       202002               0.000000   
                                       202003               0.000000   
                         골프장 운영업       201901              14.627773   
                                       201902              27.766115   
                                       201903              28.851048   
                                       201904              34.056567   
                                       201905              40.257356   
                                       201906              33.059717   
                                       201907              25.788635   
                                       201908              52.426043   
                                       201909              30.245884   
                                       201910              36.700119   
                                       201911              51.245810   
                                       201912              20.247600   
                                       202001              21.928760   
                                       202002              14.165243   
                                       202003              20.116711   
...                                                              ...   
충북           충주시         화장품 및 방향제 소매업 201901               1.198579   
                                       201902               0.391949   
                                       201903               0.410429   
                                       201904               0.000000   
                                       201905               0.047092   
                                       201906               0.235978   
                                       201907               1.019894   
                                       201908               0.100044   
                                       201909               0.047814   
                                       201910               1.175339   
                                       201911               0.000000   
                                       201912               0.000000   
                                       202001               0.000000   
                                       202002               0.000000   
                                       202003               0.000000   
                         휴양콘도 운영업      201901              57.640172   
                                       201902              54.097251   
                                       201903              36.511847   
                                       201904              65.291050   
                                       201905              58.414104   
                                       201906              62.349593   
                                       201907              44.316514   
                

In [15]:
city_sum = city_sum.set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','REG_YYMM'])

In [16]:
city_sum = city_sum.merge(new_local_sobi,left_index = True,right_index = True,how = 'left')

In [17]:
city_sum

AMT    DANGOL  \
CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM        REG_YYMM                           
강원           강릉시         건강보조식품 소매업         201901.0  2.402718e+07  1.056338   
경북           성주군         여관업                201901.0  1.273000e+06  1.434783   
             안동시         여관업                201901.0  3.638664e+07  1.421203   
             영덕군         여관업                201901.0  1.017651e+07  1.120000   
             영양군         여관업                201901.0  7.800000e+05  1.300000   
             영주시         여관업                201901.0  1.549400e+07  1.545455   
             영천시         여관업                201901.0  1.067330e+07  1.330144   
             예천군         여관업                201901.0  8.680000e+05  1.076923   
             울진군         여관업                201901.0  1.868870e+07  1.259475   
             의성군         여관업                201901.0  1.135000e+06  1.250000   
             청도군         여관업                201901.0  6.700000e+05  1.500000   
             상주시         여관업                201901.0  5.205000e+06  1.219512   
             청송군         여관업                201901.0  1.100000e+05  1.000000   
             포항시 남구      여관업                201901.0  5.138440e+07  1.339024   
             포항시 북구      여관업                201901.0  1.101883e+08  1.286950   
             성주군         여행사업               201901.0  8.900000e+04  1.000000   
             경산시         욕탕업                201901.0  1.549446e+08  1.469872   
             경주시         욕탕업                201901.0  1.282835e+08  1.504319   
             구미시         욕탕업                201901.0  1.221300e+08  1.545243   
             김천시         욕탕업                201901.0  9.461500e+06  1.443503   
             문경시         욕탕업                201901.0  1.675700e+07  1.556995   
             상주시         욕탕업                201901.0  2.214470e+07  1.489028   
             성주군         욕탕업                201901.0  8.557000e+06  1.355932   
             칠곡군         여관업                201901.0  1.840570e+07  1.470423   
             봉화군         여관업                201901.0  2.812000e+06  1.659091   
             문경시         여관업                201901.0  5.882700e+06  1.471074   
             김천시         여관업                201901.0  7.465000e+06  1.259843   
             경주시         스포츠 및 레크레이션 용품 임대업 201901.0  8.619971e+08  1.324843   
             고령군         스포츠 및 레크레이션 용품 임대업 201901.0  1.735900e+06  1.000000   
             구미시         스포츠 및 레크레이션 용품 임대업 201901.0  7.614937e+08  1.197112   
...                                                            ...       ...   
부산           북구          슈퍼마켓               202003.0  2.215864e+09  3.164664   
충남           홍성군         중식 음식점업            202003.0  6.006240e+07  1.230415   
부산           부산진구        슈퍼마켓               202003.0  3.268266e+09  2.885958   
             동구          슈퍼마켓               202003.0  6.521490e+08  2.717109   
             남구          슈퍼마켓               202003.0  2.666917e+09  3.205113   
             기장군         슈퍼마켓               202003.0  1.267011e+09  2.690873   
             금정구         슈퍼마켓               202003.0  1.909110e+09  2.931352   
             강서구         슈퍼마켓               202003.0  1.301853e+09  2.767803   
             해운대구        수산물 소매업            202003.0  6.806203e+07  1.282836   
             동래구         슈퍼마켓               202003.0  2.356261e+09  2.940802   
충남           태안군         중식 음식점업            202003.0  4.656800e+07  1.232502   
             청양군         중식 음식점업            202003.0  1.770150e+07  1.296218   
             천안시 서북구     중식 음식점업            202003.0  3.606788e+08  1.238910   
부산           해운대구        슈퍼마켓               202003.0  2.924559e+09  3.187218   
             중구          슈퍼마켓               202003.0  3.852237e+08  2.462507   
             영도구         슈퍼마켓               202003.0  7.505934e+08  2.713084   
충남           계룡시         중식 음식점업            202003.0  2.949850e+07  1.240799   
             공주시         중식 음식점업            202003.0  9.575157e

In [18]:
city_sum = city_sum.iloc[:,:-4].reset_index()
city_sum.columns =  ['CARD_SIDO_NM','CARD_CCG_NM', 'STD_CLSS_NM','REG_YYMM','AMT','DANGOL','stranger_AMT_Percent','local_AMT_Percent']

In [19]:
c = new_local_sobi.reset_index().set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])

In [20]:
c = c[c['REG_YYMM'] == 201904]

In [21]:
z = city_sum[['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM']]

In [22]:
z = z.set_index(['CARD_SIDO_NM','STD_CLSS_NM'])

In [23]:
sub = pd.read_csv('submission.csv')
sub = sub.set_index(['CARD_SIDO_NM','STD_CLSS_NM'])
sub = sub.merge(z,how = 'right',left_index = True,right_index = True)
sub = sub.reset_index().set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])
sub = sub.merge(c,left_index = True,right_index = True,how = 'left').iloc[:,:-4]
sub = sub.fillna(0).reset_index()
sub.columns = ['CARD_SIDO_NM','CARD_CCG_NM', 'STD_CLSS_NM' , 'id','REG_YYMM','AMT','a','stranger_AMT_Percent','local_AMT_Percent']
sub.drop('a',axis = 1, inplace =True)
sub = sub.drop_duplicates()

In [24]:
city_sum['MONTH'] = city_sum['REG_YYMM']%100
city_dangol = city_sum[['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','MONTH','DANGOL']]
city_dangol = city_dangol.set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','MONTH'])

In [25]:
sub['MONTH'] = sub['REG_YYMM']%100
sub = sub.set_index(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','MONTH'])
sub = sub.merge(city_dangol,left_index = True, right_index = True, how = 'left').reset_index()
sub = sub[['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'REG_YYMM', 'AMT',
       'DANGOL', 'stranger_AMT_Percent', 'local_AMT_Percent', 'MONTH','id']]

In [26]:
total = city_sum.append(sub)

In [27]:
total['Month'] = total['REG_YYMM']%100
total['Year'] = total['REG_YYMM']//100

In [28]:
total = total[['CARD_SIDO_NM','CARD_CCG_NM', 'REG_YYMM', 'STD_CLSS_NM', 'local_AMT_Percent',
       'stranger_AMT_Percent', 'Month', 'Year','DANGOL','AMT']]

In [29]:
total

,CARD_SIDO_NM,CARD_CCG_NM,REG_YYMM,STD_CLSS_NM,local_AMT_Percent,stranger_AMT_Percent,Month,Year,DANGOL,AMT
0,강원,강릉시,201901.0,건강보조식품 소매업,100.000000,0.000000,1.0,2019.0,1.056338,24027180.0
1,경북,성주군,201901.0,여관업,81.932443,18.067557,1.0,2019.0,1.434783,1273000.0
2,경북,안동시,201901.0,여관업,95.685230,4.314770,1.0,2019.0,1.421203,36386640.0
3,경북,영덕군,201901.0,여관업,83.068850,16.931150,1.0,2019.0,1.120000,10176509.0
4,경북,영양군,201901.0,여관업,100.000000,0.000000,1.0,2019.0,1.300000,780000.0
5,경북,영주시,201901.0,여관업,90.189751,9.810249,1.0,2019.0,1.545455,15494000.0
6,경북,영천시,201901.0,여관업,97.985628,2.014372,1.0,2019.0,1.330144,10673300.0
7,경북,예천군,201901.0,여관업,100.000000,0.000000,1.0,2019.0,1.076923,868000.0
8,경북,울진군,201901.0,여관업,52.358377,47.641623,1.0,2019.0,1.259475,18688700.0
9,경북,의성군,201901.0,여관업,100.000000,0.000000,1.0,2019.0,1.250000,1135000.0


In [30]:
from sklearn.preprocessing import LabelEncoder

# 인코딩
dtypes = total.dtypes
encoders = {}
for column in total.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(total[column])
        encoders[column] = encoder
        
df_num = total.copy()
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(total[column])


In [31]:
test = df_num.iloc[len(city_sum):,:]
df_train = df_num.iloc[:len(city_sum),:]

In [32]:
x_train = df_train[df_train['REG_YYMM'] != 202003].iloc[:,:-1]
y_train = np.log1p(df_train[df_train['REG_YYMM'] != 202003]['AMT'])

x_val = df_train[df_train['REG_YYMM'] == 202003].iloc[:,:-1]
y_val = np.log1p(df_train[df_train['REG_YYMM'] == 202003]['AMT'])


In [33]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'tweedie',
    'metric': 'rmse',
    'tweedie_variance_power':1.1,
    'max_depth':11,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'seed':42,
    'verbose': 0}

#[9154]	valid_0's rmse: 0.598337

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth':11,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'random_state':42}
#[7675]	valid_0's rmse: 0.596783

In [34]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [38]:
model = lgb.train(params,
                  train_ds,
                  10000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.09192
[200]	valid_0's rmse: 0.953103
[300]	valid_0's rmse: 0.889114
[400]	valid_0's rmse: 0.844933
[500]	valid_0's rmse: 0.812702
[600]	valid_0's rmse: 0.788677
[700]	valid_0's rmse: 0.769025
[800]	valid_0's rmse: 0.75182
[900]	valid_0's rmse: 0.736901
[1000]	valid_0's rmse: 0.725046
[1100]	valid_0's rmse: 0.715092
[1200]	valid_0's rmse: 0.706261
[1300]	valid_0's rmse: 0.696601
[1400]	valid_0's rmse: 0.687351
[1500]	valid_0's rmse: 0.67954
[1600]	valid_0's rmse: 0.673113
[1700]	valid_0's rmse: 0.667552
[1800]	valid_0's rmse: 0.661448
[1900]	valid_0's rmse: 0.656543
[2000]	valid_0's rmse: 0.651786
[2100]	valid_0's rmse: 0.647322
[2200]	valid_0's rmse: 0.643343
[2300]	valid_0's rmse: 0.639422
[2400]	valid_0's rmse: 0.635229
[2500]	valid_0's rmse: 0.632137
[2600]	valid_0's rmse: 0.628986
[2700]	valid_0's rmse: 0.624719
[2800]	valid_0's rmse: 0.621865
[2900]	valid_0's rmse: 0.619074
[3000]	valid_0's rmse

In [39]:
pred = model.predict(test)
pred = np.expm1(pred)

In [40]:
sub['AMT'] = pred
sub = sub.groupby(['CARD_SIDO_NM','STD_CLSS_NM','REG_YYMM','id'])['AMT'].sum().reset_index()
sub = sub[['id','REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']].sort_values(by = 'id')
sub

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,6.644355e+07
2,1,202004,강원,골프장 운영업,1.741038e+09
4,2,202004,강원,과실 및 채소 소매업,6.719036e+08
6,3,202004,강원,관광 민예품 및 선물용품 소매업,1.553057e+07
8,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.411964e+05
10,5,202004,강원,그외 기타 스포츠시설 운영업,3.266471e+06
12,6,202004,강원,그외 기타 종합 소매업,3.850943e+08
14,7,202004,강원,기타 대형 종합 소매업,6.508869e+09
16,8,202004,강원,기타 수상오락 서비스업,1.819544e+06
18,9,202004,강원,기타 외국식 음식점업,8.396905e+08


In [41]:
h = pd.read_csv('submission.csv')
h.set_index('id',inplace = True)

In [42]:
l = set(sub['id'])
lst2 = []
for i in range(1393):
    if i not in l:
        lst2.append(i)

In [43]:
sub = sub.append(h.loc[lst2].reset_index())
sub = sub.sort_values(by = 'id').reset_index().drop('index',axis =1)

In [44]:
sub.set_index('id').to_csv('jeju_final_tweedie.csv',encoding = 'utf-8-sig')

In [45]:
def set_sumit(submission, pre_df):
    
    from sklearn.metrics import mean_squared_log_error
    
    pre_df= pre_df[pre_df['REG_YYMM'] == 202004]
    submission= submission[submission['REG_YYMM'] == 202004]
    
    pre_df.loc[pre_df.CARD_SIDO_NM == '제주','weight'] = 3
    pre_df.loc[pre_df.CARD_SIDO_NM != '제주','weight'] = 1
    
    weight = pre_df['weight'].values
    
    all_amt = np.sqrt(mean_squared_log_error(submission['AMT'],pre_df['AMT'],sample_weight=weight))
    return all_amt

In [46]:
only4 = pd.read_csv('only_4.csv')

In [47]:
set_sumit(only4,sub)

3.3500896176337087